In [1]:
from decimal import Decimal, ROUND_HALF_UP

In [2]:
def round_dec(n, d=2):
    s = '0.' + '0' * d
    return Decimal(str(n)).quantize(Decimal(s), rounding=ROUND_HALF_UP)

## 等额本金
每月还款额Fm=每月还款本金+每月还款利息。Pm+Rm

每月还款本金Pm=贷款总额/贷款月数 P/M。

每月还款利息Rm=贷款本金余额 * 贷款月利率 Pb * i（ 贷款月利率=年利率/12）。

贷款本金余额Pb=贷款总额-已还款月数 * 每月还款本金 P-m * Pm

In [3]:
def average_capital(P, I, M):
    '''
    等额本金
    返回一个包含每月返款数据元组的迭代器
    '''
    Pm = Decimal(P) / M  # 每月返本金，每月相等
    i = Decimal(I) / 12  # 月利率
    for m in range(M):
        Pb = P - m * Pm  # 月初本金余额
        Rm = Pb * i  # 每月返收益
        Fm = Pm + Rm  # 每月返款额
        Pe = Pb - Pm  # 月末本金余额
        result = (m + 1, Fm, Pm, Rm, Pb, Pe)
        yield result

## 等额本息
本月还息=本月余额×月利率

本月还本=每月还款额-本月还息

每月还款额=[贷款本金×月利率×（1+月利率）^还款月数]÷[（1+月利率）^还款月数－1]

In [4]:
def average_capital_plus_interest(P, I, M):
    '''
    等额本息
    返回一个包含每月返款数据元组的迭代器
    '''
    P = Decimal(P)
    i = Decimal(I) / 12  # 月利率
    result = ()
    Fm = P * i * ((1 + i)**M) / (((1 + i)**M) - 1)  # 每月返款额，每月相等
    for m in range(M):
        if not result:  # 此处result为上月数据
            Pb = P  # 月初余额，若上月数据不存在则为首月，月初余额为本金
        else:
            Pb = result[-1]  # 月初余额，为上月月末余额
        Rm = Pb * i  # 还息
        Pm = Fm - Rm  # 还本
        Pe = Pb - Pm  # 月末余额
        result = (m + 1, Fm, Pm, Rm, Pb, Pe)
        yield result

In [5]:
def calculate(P, I, M, func, detail=True):
    '''
    计算并输出投资收益结果
    args:
        P:投资金额
        I:年收益率
        M:投资期数（月）
        func:投资模式:
                "ac":等额本金
                "acpi":等额本息
        detail:boolean,是否打印每期明细
    ----------------------------
    return:None
    '''
    func_dict = {'ac': average_capital, 'acpi': average_capital_plus_interest}
    f = func_dict[func]
    F = Decimal(0)
    R = Decimal(0)
    for r in f(P, I, M):
        F += r[1]
        R += r[3]
        r = r[0:1] + tuple(map(round_dec, r[1:]))  # 对元组中数字部分四舍五入
        if detail: print('第%d月返款额：%s，返本金：%s，返收益，%s，月初本金余额：%s，月末本金余额：%s' % r)
    print('返款总额：%s，收益总计：%s' % (round_dec(F), round_dec(R)))

In [6]:
calculate(10000, 0.05, 6, 'acpi')

第1月返款额：1691.06，返本金：1649.39，返收益，41.67，月初本金余额：10000.00，月末本金余额：8350.61
第2月返款额：1691.06，返本金：1656.26，返收益，34.79，月初本金余额：8350.61，月末本金余额：6694.35
第3月返款额：1691.06，返本金：1663.16，返收益，27.89，月初本金余额：6694.35，月末本金余额：5031.18
第4月返款额：1691.06，返本金：1670.09，返收益，20.96，月初本金余额：5031.18，月末本金余额：3361.09
第5月返款额：1691.06，返本金：1677.05，返收益，14.00，月初本金余额：3361.09，月末本金余额：1684.04
第6月返款额：1691.06，返本金：1684.04，返收益，7.02，月初本金余额：1684.04，月末本金余额：-0.00
返款总额：10146.34，收益总计：146.34


In [7]:
calculate(10000, 0.05, 6, 'ac', False)

返款总额：10145.83，收益总计：145.83


## 等额本息-复投同利率等额本息 

In [8]:
def bxft(M, P, I, m=0, G=0):
    i = I / 12
    Fm = P * i * ((1 + i)**(M - m)) / (((1 + i)**(M - m)) - 1)
    # print('第%s月投资金额：%s，投资单期回款：%s，投资期末收益：%s' % (m+1, P, Fm, Fm*(M-m)-P))
    G = G + Fm * (M - m) - P
    m += 1
    if m < M and m > 1:
        return bxft(M, Fm + P, I, m, G)
    elif m <= 1:
        return bxft(M, Fm, I, m, G)

    # print('最终收益：%s' % G)
    return G

In [9]:
bxft(6, 10000, 0.085)

432.5974996289442

## 收益分析 

In [10]:
G = []
for m in np.arange(3, 37, 3):
    G.append(bxft(M=m, I=0.085, P=10000))

NameError: name 'np' is not defined

In [ ]:
data = pd.DataFrame({'期数': np.arange(3, 37, 3), '收益': G})
data

In [ ]:
data['最终收益率'] = data['收益'] / 10000

In [ ]:
data['年化收益率'] = (12 / data['期数']) * data['收益'] / 10000